In [212]:
import os
import json
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import pandas as pd

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from neo4j import GraphDatabase

In [5]:
DB_ULR = "neo4j://localhost:7687"
DB_USER = "neo4j"
DB_PASS = "test1234"
DB_NAME = "android"
DB_ENCRYPTED = False


In [6]:
driver = GraphDatabase.driver(DB_ULR, auth=(DB_USER, DB_PASS), encrypted=DB_ENCRYPTED)

In [74]:
# Schema 
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run("CREATE CONSTRAINT IF NOT EXISTS ON (n:App) ASSERT (n.name) IS NODE KEY").consume()
    )
    result = session.write_transaction( lambda tx: 
        tx.run("CREATE CONSTRAINT IF NOT EXISTS ON (n:Package) ASSERT (n.name) IS NODE KEY").consume()
    )
    result = session.write_transaction( lambda tx: 
        tx.run("CREATE INDEX IF NOT EXISTS FOR (p:Package) ON (p.louvain)").consume()
    )

In [63]:
df = pd.read_csv("/Users/haklof/neo4j-enterprise/import/Android_Permission.csv").dropna()
df.head()

,App,Package,Category,Description,Rating,Number of ratings,Price,Related apps,Dangerous permissions count,Safe permissions count,...,Your personal information : read calendar events (D),Your personal information : read contact data (D),Your personal information : read sensitive log data (D),Your personal information : read user defined dictionary (D),Your personal information : retrieve system internal state (S),Your personal information : set alarm in alarm clock (S),Your personal information : write Browser's history and bookmarks (D),Your personal information : write contact data (D),Your personal information : write to user defined dictionary (S),Class
0,Canada Post Corporation,com.canadapost.android,Business,Canada Post Mobile App gives you access to som...,3.1,77,0.0,"{com.adaffix.pub.ca.android, com.kevinquan.gas...",7.0,1,...,0,1,0,0,0,0,0,1,0,0
1,Word Farm,com.realcasualgames.words,Brain & Puzzle,Speed and strategy combine in this exciting wo...,4.3,199,0.0,"{air.com.zubawing.FastWordLite, com.joybits.do...",3.0,2,...,0,0,0,0,0,0,0,0,0,0
2,Fortunes of War FREE,fortunesofwar.free,Cards & Casino,"Fortunes of War is a fast-paced, easy to learn...",4.1,243,0.0,"{com.kevinquan.condado, hu.monsta.pazaak, net....",1.0,1,...,0,0,0,0,0,0,0,0,0,0
3,Better Keyboard: Avatar Purple,com.cc.betterkeyboard.skins.avatarpurple,Libraries & Demo,Skin for Better Keyboard featuring a glossy fe...,3.6,2589,0.0,{eu.gdumoulin.betterandroid.skins.transparent....,0.0,0,...,0,0,0,0,0,0,0,0,0,0
5,Ms Claus Live Wallpaper,tmc.christmaslady.livewallpaper,Personalization,Ms Claus Live Wallpaper<p>Find more Free apps ...,4.5,16,0.0,"{tmc.christmassanta.livewallpaper, tmc.winterh...",3.0,1,...,0,0,0,0,0,0,0,0,0,0


In [64]:
df.App.value_counts()

Tic Tac Toe               45
????????????              24
Slot Machine              16
??????????????????        15
?????????????????????     14
                          ..
Vienna                     1
Adult Sexy Strip Games     1
Montreal Bixi              1
AL Bubble Blast            1
Book Tower Madness         1
Name: App, Length: 22176, dtype: int64

In [93]:
apps = list()
packages = set()
app_installs = list()
app_links = list()
for idx, row in df.iterrows():
    app = { 
        'name': row['App'],
        'description': row['Description'],
        'rating': row['Rating'],
        'numberOfRatings': row['Number of ratings'],
        'price': row['Price'],
        'dangerous': row['Dangerous permissions count'],
        'safe': row['Safe permissions count'],
        'rowNum': idx,
        'class': row['Class']
    }
    if row['Package'] != None:
        app_installs.append({'app': row['App'], 'package': row['Package']})
    app_links.append({'app': row['App'],  "relatedPackages": [ package.strip() for package in row['Related apps'][1:-1].split(',') ]})
    packages.add(row['Package'])
    [ packages.add(package.strip()) for package in row['Related apps'][1:-1].split(',') ]
    apps.append(app)        
print(apps[0])

{'name': 'Canada Post Corporation', 'description': 'Canada Post Mobile App gives you access to some of the the most popular tools on <a href="http://www.google.com/url?q=http://canadapost.ca&usg=AFQjCNGLsUtAzxPlFZBfYYGjv6mnRiMZJw" target="_blank">canadapost.ca</a>.  You can now Find a Postal Code, create your own personalized stamps and postcards with Picture Postage, Track a Package and Find a Post Office anywhere in Canada.<p>Now introducing Picture Postage Postcard!<p>Turn your photos into printed postcards???with delivery anywhere in the world for just $2.45 per card! Take a photo or select one from your camera roll, choose the recipient(s) from your contacts list and write your personal message. We???ll print, stamp and mail the postcard to each person on your list!<p>Use Picture Postage to create a personalized stamp for your letters and invitations.  Get started by taking a picture with your device, or access any photos that you have stored on your camera roll!<p>Use Find a Post

In [77]:
# Create Package nodes
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run(
            """
            UNWIND $packages as package
            MERGE (a:Package{name:package})
            """,
            packages = list(packages)
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

""


In [144]:
# Create App nodes
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run(
            """
            UNWIND $apps as app
            MERGE (a:App{name:app.name})
            SET a.description = app.description,
                a.rating = toFloat(app.rating),
                a.numberOfRatings = toInteger(app.numberOfRatings),
                a.price = toFloat(app.price),
                a.dangerous = toInteger(app.dangerous),
                a.safe = toInteger(app.safe),
                a.rowNum = app.rowNum,
                a.class = toInteger(app.class)
            """,
            apps = apps
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

""


In [145]:
# Create App - INSTALLS -> Package rels
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run(
            """
            UNWIND $app_installs as rel
            MATCH (a:App{name: rel.app}), (p:Package{name: rel.package})
            MERGE (a)-[:INSTALLS]->(p)
            """,
            app_installs = app_installs
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

""


In [146]:
# Create App - LINKS -> Package rels
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run(
            """
            UNWIND $app_links as rel
            MATCH (a:App{name: rel.app}), (p:Package)
            WHERE p.name in rel.relatedPackages
            MERGE (a)-[:LINKS]->(p)
            """,
            app_links = app_links
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

""


In [147]:
# Add label for Malware
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run(
            """
            MATCH (a:App) where a.class = 1
            set a:Malware
            """
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

""


In [148]:
# Calculate dangerousness of app
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run(
            """
            MATCH (a:App)
            set a.dangerousness = a.dangerous/(a.safe + a.dangerous + 0.00001)
            """
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

""


In [193]:
# Is an app more safe if other appls link what it installs?
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run(
            """
            MATCH (a:App)-[:INSTALLS]->(p:Package)
            set a.numberOfDependants = size( (p)<-[:INSTALLS]-() )
            """
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

""


In [194]:
# Create graph projection 
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run("CALL gds.graph.drop('app-projection', false)").consume()
    )
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run(
            """
            CALL gds.graph.create.cypher('app-projection',
            'MATCH (a:App) return id(a) as id, 
                ["App"] as labels,
                a.class as class, 
                coalesce(a.numberOfRatings, 0) as numberOfRatings, 
                coalesce(a.rating,0.0) as rating, 
                coalesce(a.price,0.0) as price,
                coalesce(a.dangerous,0) as dangerous,
                coalesce(a.safe,0) as safe,
                a.dangerousness as dangerousness,
                a.numberOfDependants as numberOfDependants',
            'MATCH (a1:App)-[:LINKS|INSTALLS]->(p:Package)<-[:LINKS|INSTALLS]->(a2:App) 
            RETURN id(a1) as source, id(a2) as target, (a1.dangerousness+a2.dangerousness)/2 as risk
            ')
            """
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

,nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,createMillis
0,"MATCH (a:App) return id(a) as id, \n ...",MATCH (a1:App)-[:LINKS|INSTALLS]->(p:Package)<...,app-projection,22176,444070,734


In [195]:
# Pagerank
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run(
            """
            call gds.pageRank.mutate('app-projection',{
                    maxIterations: 100,
                    mutateProperty: 'pageRank'
            })    
            """
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

,mutateMillis,nodePropertiesWritten,ranIterations,didConverge,centralityDistribution,postProcessingMillis,createMillis,computeMillis,configuration
0,0,22176,97,True,"{'p99': 2.3757314682006836, 'min': 0.149999618...",93,0,600,"{'maxIterations': 100, 'relationshipWeightProp..."


In [196]:
# WCC
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run(
            """
            call gds.wcc.mutate('app-projection',{
                    mutateProperty: 'wcc'
            })    
            """
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

,mutateMillis,nodePropertiesWritten,componentCount,componentDistribution,postProcessingMillis,createMillis,computeMillis,configuration
0,0,22176,414,"{'p99': 5, 'min': 1, 'max': 21631, 'mean': 53....",3,0,9,"{'seedProperty': None, 'consecutiveIds': False..."


In [197]:
# Create embedding
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run(
            """
            call gds.fastRP.mutate('app-projection',{
                embeddingDimension: 8, 
                relationshipWeightProperty: 'risk',
                iterationWeights: [0.0, 1.0, 0.8, 0.7],
                mutateProperty: 'embedding'
                })
                
            """
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

,nodePropertiesWritten,mutateMillis,nodeCount,createMillis,computeMillis,configuration
0,22176,0,22176,0,26,"{'normalizationStrength': 0.0, 'iterationWeigh..."


In [226]:
with driver.session(database = DB_NAME) as session:
    try: 
        result = session.write_transaction( lambda tx: 
            tx.run("call gds.beta.model.drop('risk-model')").consume()
        )
    except:
        print("Failed to remove model")
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        # Todo: 'numberOfRatings' has NaN value?
        tx.run(
            """
            call gds.alpha.ml.nodeClassification.train('app-projection',{
                nodeLabels: ['App'],
                modelName: 'risk-model',
                featureProperties: ['embedding', 'rating', 'price', 'dangerous', 'safe', 'dangerousness', 'numberOfDependants', 'pageRank', 'wcc'], 
                targetProperty: 'class', 
                metrics: ['F1_WEIGHTED', 'ACCURACY'], 
                holdoutFraction: 0.7, 
                validationFolds: 5, 
                randomSeed: 3,
                params: [
                    {penalty: 0, minEpochs: 10, maxEpochs: 10000, tolerance: 0.00001, patience: 5},
                    {penalty: 0.5, minEpochs: 10, maxEpochs: 10000, tolerance: 0.00001, patience: 5},
                    {penalty: 1.0, minEpochs: 10, maxEpochs: 10000, tolerance: 0.00001, patience: 5}
                    ]
            }) yield modelInfo
            return modelInfo
            """
        ).data()
    )
    print(json.dumps(result, indent=2))

[
  {
    "modelInfo": {
      "name": "risk-model",
      "metrics": {
        "F1_WEIGHTED": {
          "outerTrain": 0.601520248676009,
          "test": 0.5907851508803652,
          "validation": [
            {
              "min": 0.5894410013625989,
              "avg": 0.606261464782155,
              "params": {
                "maxEpochs": 10000,
                "minEpochs": 10,
                "penalty": 0.0,
                "patience": 5,
                "batchSize": 100,
                "sharedUpdater": false,
                "tolerance": 1e-05,
                "concurrency": 4
              },
              "max": 0.6242673719760728
            },
            {
              "min": 0.5903824053568921,
              "avg": 0.6038887721555357,
              "params": {
                "maxEpochs": 10000,
                "minEpochs": 10,
                "penalty": 0.5,
                "patience": 5,
                "batchSize": 100,
                "sharedUpdater": false,
